# Tensorflow 실습 2

python의 딥러닝 라이브러리인 Tensorflow에 대해 알아보자.

## 간단한 방식의 딥러닝 구현 

### MNIST 데이터셋

<table>
  <tr><td>
    <img src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png" width="600">
  </td></tr>
  <tr><td align="center">
    <b>그림 1.</b> MNIST 샘플 이미지 <br/>&nbsp;
  </td></tr>
</table>

- 머신 러닝에서 사용되는 가장 기본적인 데이터 셋
- 손글씨로 쓰여진 0~9의 숫자들이 28*28 pixel 크기의 흑백 이미지 데이터로 저장되어 있음
- 주어진 이미지가 0~9 중 어떤 숫자인지 분류하는 model 을 만드는 것이 목표

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

# mnist dataset의 tf.keras.datasets.mnist로 쉽게 불러올 수 있음
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# x는 이미지, y는 0부터 9까지 각 이미지에 맞는 label
print(x_train[0].shape)
print(x_train[0].dtype)
print(y_train[:5])
print(y_train[0])
plt.imshow(x_train[0], cmap='Greys')
plt.show()

# training data로 60000개의 image, test data로 10000개의 image 사용
print(len(x_train))
print(len(x_test))

### 모델 선언 및 학습, 평가

- tf.keras.models.Sequential 안에 tf.keras.layers의 layer들로 모델을 구성
작성한 순서대로 input에 각 layer가 적용됨
- Flatten : 28 * 28의 tensor를 784 size의 tensor로 펴주는 역할
- Dense : 임의의 size의 input을 특정한 size로 mapping 시켜주는 fully connected layer
- activation function으로 relu, softmax 등을 사용할 수 있음
- Dropout : dropout layer의 input에서 각각의 값에 대해, 일정 확률로 0값을 배정함
  - 학습 시에만 적용되며, test 할때는 적용되지 않음
  - overfitting을 막아주는 regularizer 역할



In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

# model.complile에서는 optimizer, loss, metric을 지정
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())

- model.fit을 통해, 학습 데이터를 이용하여 학습함
  - validation_split=0.2 : 학습 데이터 중 80%는 학습에 이용하고, 20%는 매 epoch 마지막에 validation에 이용함
  - epoch=5 : 학습 데이터를 총 5번 활용하여 학습함
  - verbose=2 : 학습 log의 verbosity를 정하는 값
    - verbose = 1 > progress bar and one line per epoch
    - verbose = 0 > silent
    - verbose = 2 > one line per epoch

- model.evaluate에서 학습된 모델을 이용하여, test 데이터에 대한 성능을 확인

In [ ]:
model.fit(x_train, y_train, validation_split=0.2, epochs=5, verbose=1)

loss, accuracy = model.evaluate(x_test, y_test)
print("test loss : %.3f, test accuracy : %.3f" % (loss, accuracy))

- 학습된 모델을 이용해서, 이미지에 대한 분류 결과를 출력하는 방법

In [ ]:
y_pred = model.predict(x_test)
print(y_pred[:4])
print(tf.argmax(y_pred[:4], axis=-1))

plt.imshow(x_test[0], cmap='Greys')
plt.show()
plt.imshow(x_test[1], cmap='Greys')
plt.show()

## tf.data.Dataset
- 일반적으로 데이터를 input으로 사용하기 위해 필요한 절차 (Extract, Transform, Load)
  - Extract: 파일로 저장된 데이터를 불러옴 (csv file, numpy file, tfrecord file 등)
  - Transform: 원하는 방식으로 전처리하거나, augmentation을 적용해야 하고(이미지 데이터 rotation, 텍스트 데이터 벡터화, 오디오 데이터의 signal process 등), random하게 batch를 구성
  - Load: transformed data를 GPU/TPU (accelerator devices)에 올려서 연산

- 위와 같이 복잡한 input pipeline을 간단하게 처리할 수 있도록 지원하는 API



- 가상의 데이터인 x, y 생성
  - x는 총 10개의 2 x 3 data
  - y는 0부터 9까지 총 10개의 integer
  - 총 10개의 x, y가 짝을 이루는 dataset






In [ ]:
x = tf.random.normal([10, 2, 3])
y = [i for i in range(10)]

dataset = tf.data.Dataset.from_tensor_slices((x, y))

batch_size = 4

print("-----------batched dataset----------- \n")
# dataset.batch() function을 이용하여 dataset을 batch 단위로 나눌 수 있음
# batch size 4의 의미 : 전체 data를 4개씩 묶어서 불러옴
# tensorflow의 dataset은 for문을 통해 data를 불러올 수 있음
batched_dataset = dataset.batch(batch_size)
for batch in batched_dataset:
    print("start of batch")
    x, y = batch
    print("x : ", x)
    print("y : ", y, "\n")

- 위 출력 결과의 문제점
  - 전체 data가 순차적으로 구성
  - 학습 데이터는 random하게 batch를 구성하는 것이 필요


- 1 epoch : 전체 데이터를 한번 이용하는 것

- dataset을 shuffle한 후, batch를 나누는 방법: 자동으로 epoch 마다 shuffle
  - buffer_size 만큼 순서대로 가져온 후, 그 안에서 shuffle 후 batch 구성
  - shuffle과 batch의 순서도 중요함
  - 20000장의 이미지가 있는데, 1번에서 10000번 까지는 고양이 이미지, 10001번부터 20000번 까지는 고양이가 아닌 이미지라고 하자.
    - 이 때 buffer_size=1000 이면 일단 1000개의 고양이 이미지를 가져온 후 shuffle, 즉 고양이로만 구성된 batch들을 구성하게 됨
    - 학습이 제대로 이루어지지 않음
  - buffer_size가 1이라면 섞이지 않음
  - 전체 data를 완전히 random하게 만들기 위해서는 buffer_size가 전체 data 개수와 같거나 크게 설정

- 아래 실행 결과를 확인하면, 각 batch의 구성과 data의 순서가 random하게 바뀐 것을 확인할 수 있음 

In [ ]:
epochs = 2
buffer_size = 10

print("-----------shuffled dataset----------- \n")
shuffled_datatset = dataset.shuffle(buffer_size).batch(batch_size)
for epoch in range(epochs):
    print("start of epoch: ", epoch, "\n")
    for batch in shuffled_datatset:
        print("start of batch")
        x, y = batch
        print("x : ", x)
        print("y : ", y, "\n")


### Transform을 위해, map 함수 활용 
- tf.data.Dataset에서 각각의 데이터에 대해 함수를 일관되게 적용하고 싶은 경우, dataset.map 함수를 이용함
- 결과를 보면, 기존 음수였던 값이 절대값 처리되어 양수로 변환된 것을 확인




In [ ]:
# tf.abs()를 input에 적용하는 함수
def abs_function(x):
    x = tf.abs(x)
    return x

print("-----------dataset applied absolute function----------- \n")

# lambda x, y의 의미: 모든 x, y에 대해 오른쪽과 같은 함수를 적용하라는 의미
abs_dataset = dataset.map(lambda x, y: (abs_function(x), y))

# take()는 정해진 개수의 데이터를 가져오도록 하는 함수
for x, y in abs_dataset.batch(4).take(1):
    print("x : ", x)
    print("y : ", y, "\n")


(참고) tensorflow의 함수를 활용하지 않으면서, map 함수를 활용하는 방법
- 이전 예시에서는 tf function을 이용하였으므로, dataset map 함수가 정상적으로 작동
- 그러나 data에 전처리 또는 augmentation 할 때, tensorflow 함수가 아닌 별도의 패키지를 필요로하는 경우가 많음
- 앞에서와 동일한 구조로, np.abs를 활용하는 함수를 적용하면 아래와 같은 오류가 발생함
- Tensor를 numpy array로 변환할수 없다는 의미



In [ ]:
import numpy as np

def numpy_abs_function(x):
    x = np.abs(x)
    return x

try:
    print("-----------dataset applied numpy absolute function----------- \n")
    abs_dataset = dataset.map(lambda x, y: (numpy_abs_function(x), y))
    for x, y in abs_dataset.batch(4).take(1):
        print("x : ", x)
        print("y : ", y, "\n")
except:
    print("=====Failure to use numpy function for dataset map function")


- tf.py_function : 일반적인 python function을 tensorflow에 알맞은 function으로 변환
  - input으로 함수, input list, output type을 넣어줌
- 위 함수를 이용하여 변환된 함수를 dataset map에 적용
- 오류 없이 작동하는 것을 확인


In [ ]:
# tf.py_function으로 tensorflow 외부 함수를 사용할 수 있게 변환
def tf_numpy_abs_function(x):
    tf_float = tf.py_function(
        numpy_abs_function,
        [x],
        tf.float32
    )
    return tf_float

print("-----------dataset applied numpy absolute function----------- \n")
abs_dataset = dataset.map(lambda x, y: (tf_numpy_abs_function(x), y))
for x, y in abs_dataset.batch(4).take(1):
    print("x : ", x)
    print("y : ", y, "\n")

##Tensorboard

- Tensorboard는 Tensorflow의 시각화 도구
- 측정값을 시각화 하거나 모델의 연산 Graph를 시각화 하는 도구를 제공함  
- 여기서는 uniform 분포를 따르는 값을 sampling하여, 1000step 동안 값을 저장함



In [ ]:
# Log file을 저장할 path를 설정
tb_path = 'tensorboard'
# summary_writer를 만들고, log 값을 저장하고 싶은 경우에 활용함
summary_writer = tf.summary.create_file_writer(tb_path)
with summary_writer.as_default():
    for step in range(1000):
        # tf.summary.scalar는 tag, 값, step을 인자로 받음
        tf.summary.scalar('random uniform [0,1]', tf.random.uniform([1])[0], step=step)

- 연산 그래프를 시각화 하는 방법
- 복잡한 딥러닝 모델의 연산도 그래프로 시각화 가능

In [ ]:
# 연산 추적하도록 설정
tf.summary.trace_on(graph=True, profiler=True)

# 연산하는 함수를 graph mode로 설정 (Autograph)
# Autograph: 함수 위에 @tf.function 데코레이터를 사용해주면, 함수 안의 연산을 자동으로 그래프 모드로 수행함
@tf.function
def opertation(x, y):
    x = tf.tanh(x)
    z = x + y
    return z

# 연산 실행
x = tf.random.normal([1])
y = tf.random.uniform([1])
z = opertation(x, y)

# 연산을 추적한 결과를 log file로 저장
with summary_writer.as_default():
    tf.summary.trace_export(name="trace", step=0, profiler_outdir=tb_path)

# 연산 추적 off
tf.summary.trace_off()

- 저장한 값을 시각한 결과

In [ ]:
# Colab 환경에서 tensorboard를 사용하는 방법
%load_ext tensorboard
%tensorboard --logdir tensorboard/

- Colab이 아닌 일반 컴퓨터에서 tensorboard를 활용하는 방법
  - cmd 창을 실행
  - 명령어 입력: tensorboard –-logdir=(로그파일 저장 경로)
  - 이후 노출되는 링크를 웹브라우저에 붙여넣으면 tensorboard 확인 가능
  - 일반적인 local 환경에서는 다음 링크를 사용 : http://localhost:6006/